<a href="https://colab.research.google.com/github/IllgamhoDuck/Quantum-Circuit-Optimization-with-Deep-learning/blob/master/Optimized_circuit_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Optimized Circuit Predictor
**Predict optimized quantum circuit generated by level 3 transpiler passes using an LSTM seq2seq deep learning model**

### Setting environment
- package install
- Google Drive mount
- path configuration


In [0]:
!pip install --quiet qiskit nxpd

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# TODO - modify path to fit your computer
# dataset_path - where to save the dataset including circuit information
# model_path = where to save the deep learning model you created
# weight_path = Where to save the model weight you trained

dataset_path = '/content/drive/My Drive/QUANTUM/qaward/dataset'
model_path = '/content/drive/My Drive/QUANTUM/qaward/model and weight'
weight_path = '/content/drive/My Drive/QUANTUM/qaward/model and weight'

### Train step 0 - load the total dataset
- Load dataset
- Divide it to Train / Validation / Test (default 70% / 10% / 20%)
- check the shape of data

In [0]:
import os
import pickle

# TODO - Choose your dataset name
dataset_name = "5_qubit_encoded_dataset.pkl"
with open(os.path.join(dataset_path, dataset_name), 'rb') as f:
    total_dataset = pickle.load(f)

**Store dataset as a global variable**

In [0]:
for k, v in total_dataset.items():
    print(k)
    exec("{} = v".format(k))

enc_in
enc_out_type
enc_out_gate
enc_out_control
enc_out_target
enc_out_classic
enc_out_angle
dec_in
dec_out_concatenated
dec_out_type
dec_out_gate
dec_out_control
dec_out_target
dec_out_classic
dec_out_angle
layout_out
hardware_in
coupling_map
u1_error
u2_error
u3_error
cx_error
readout_error


**Devide dataset to `Train / Validation / Test` set**

    Train - 70%
    Validation - 10%
    Test - 20%

In [0]:
import numpy as np

data_len = enc_in.shape[0]
# TODO - choose the ratio
# Test ratio will be choosed by train and validation ratio
# 0.7 means 70% if dataset is 10,000 it will be 7,000
train_ratio = 0.7
validation_ratio = 0.1

# this ratio will be used on the below code
# first ratio - fr / second ratio - sr
fr = train_ratio
sr = train_ratio + validation_ratio

# Generate input
train_enc_in = enc_in[:round(data_len*fr)]
validation_enc_in = enc_in[round(data_len*fr):round(data_len*sr)]
test_enc_in = enc_in[round(data_len*sr):]

# Generate hareware information -> Noise and topologie input
train_hardware_in = hardware_in[:round(data_len*fr)]
validation_hardware_in = hardware_in[round(data_len*fr):round(data_len*sr)]
test_hardware_in = hardware_in[round(data_len*sr):]

# Generate output decoder input
train_dec_in = dec_in[:round(data_len*fr)]
validation_dec_in = dec_in[round(data_len*fr):round(data_len*sr)]
test_dec_in = dec_in[round(data_len*sr):]

# Generate output decoder output
train_dec_out = dec_out_concatenated[:round(data_len*fr)]
validation_dec_out = dec_out_concatenated[round(data_len*fr):round(data_len*sr)]
test_dec_out = dec_out_concatenated[round(data_len*sr):]

# decode type output
train_type = dec_out_type[:round(data_len*fr)]
validation_type = dec_out_type[round(data_len*fr):round(data_len*sr)]
test_type = dec_out_type[round(data_len*sr):]

# decoder output
# gate
train_gate = dec_out_gate[:round(data_len*fr)]
validation_gate = dec_out_gate[round(data_len*fr):round(data_len*sr)]
test_gate = dec_out_gate[round(data_len*sr):]

# control
train_control = dec_out_control[:round(data_len*fr)]
validation_control = dec_out_control[round(data_len*fr):round(data_len*sr)]
test_control = dec_out_control[round(data_len*sr):]

# target
train_target = dec_out_target[:round(data_len*fr)]
validation_target = dec_out_target[round(data_len*fr):round(data_len*sr)]
test_target = dec_out_target[round(data_len*sr):]

# classic
train_classic = dec_out_classic[:round(data_len*fr)]
validation_classic = dec_out_classic[round(data_len*fr):round(data_len*sr)]
test_classic = dec_out_classic[round(data_len*sr):]

# classic
train_angle = dec_out_angle[:round(data_len*fr)]
validation_angle = dec_out_angle[round(data_len*fr):round(data_len*sr)]
test_angle = dec_out_angle[round(data_len*sr):]

In [0]:
print(train_enc_in.shape)
print(train_hardware_in.shape)
print(train_dec_in.shape)
print(train_dec_out.shape)

(21000, 65, 29)
(21000, 70)
(21000, 131, 29)
(21000, 131, 29)


### Train step 1 - Create Model
- Choose model hyperparamter
- Choose the output style


    - One Concatenated output (mse)
    - type (binary crossentropy) / angle (mse)
    - each types (categorical crossentropy) / angle (mse)

- Create LSTM seq2seq model for circuit optimization
- Check the model

In [0]:
import numpy as np
import keras
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import Input, LSTM, RepeatVector, LeakyReLU, ReLU
from keras.layers import BatchNormalization, concatenate
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.optimizers import SGD, RMSprop, Adam, Adagrad, Adadelta
from keras import objectives
from keras.callbacks import ModelCheckpoint
from keras.models import model_from_json

Using TensorFlow backend.


In [0]:
# Model hyperparmeter
# TODO - choose the latent dimension
# This indicates the circuit latent space (manifold)
latent_dim = 32

circuit_dim = enc_in.shape[2]
circuit_timesteps = enc_in.shape[1]
env_dim = hardware_in.shape[1]
decoder_timesteps = dec_out_concatenated.shape[1]
layout_n = layout_out.shape[1]

In [0]:
def optimize_circuit(circuit_dim,
                     circuit_timesteps,
                     env_dim,
                     latent_dim,
                     decoder_timesteps,
                     layout_n,
                     output_style=0):
    """
    output_style
    0 - concatenated
        (gate + control + target + classic + angle)
        [29]
    1 - type + angle
        (gate + control + target + classic) (angle)
        [26] + [3]
    2 - Seperated   
        (gate) (control) (target) (classic) (angle)
        [8] + [6] + [6] + [6] + [3]
    """
    # Encode the circuit to manifold
    circuit_input = Input(shape=(circuit_timesteps, circuit_dim), name='circuit_encode')
    encoded = LSTM(64)(circuit_input)
    # encoded = Dense(64)(encoded)
    # encoded = LeakyReLU(alpha=0.05)(encoded)
    # encoded = Dense(64)(encoded)
    # encoded = LeakyReLU(alpha=0.05)(encoded)

    # z = Dense(latent_dim)(encoded)
    # z = LeakyReLU(alpha=0.05)(z)

    # Hardware information include hardware topologie and noise information
    hardware_input = Input(shape=(env_dim,), name='hardware')

    # latent_dim (32) + hardware (70) = 132
    # Concatenate z and hardware information
    concatenated = concatenate([encoded, hardware_input])

    # Dense layer
    x = Dense(128)(concatenated)
    x = Dense(128, activation='relu')(x)
    x = Dense(128, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    x = Dense(32)(x)
    x = BatchNormalization()(x)

    # Predict layout
    layout = Dense(layout_n, activation='sigmoid')(x)

    # Concatenate z and hardware and layout information to predict the optimized circuit
    # latent_dim (32) + hardware (70) + layout(5) = 137
    circuit_info = concatenate([concatenated, layout])

    # Dense layer
    circuit_info = Dense(64, activation='relu')(circuit_info)
    circuit_info = Dense(circuit_dim, activation='relu')(circuit_info)

    # LSTM h, c
    hc = [circuit_info, circuit_info]

    # Decoder input
    decoder_input = Input(shape=(decoder_timesteps, circuit_dim), name='decoder_input')

    # Decoder LSTM
    decoder = LSTM(circuit_dim, return_sequences=True, return_state=True)
    decoder_output, h, c = decoder(decoder_input, initial_state=hc)

    # Process output
    # decoder output = (batch_size, decoder_timestep, circuit_dim)
    # (batch_size, 106, 33)

    # 1. Gate type (8) 
    gate = Dense(8, activation='softmax', name='gate')(decoder_output)
    
    # 2. Qubit / classical type (18)
    control = Dense(6, activation='softmax', name='control')(decoder_output)
    target = Dense(6, activation='softmax', name='target')(decoder_output)
    classic = Dense(6, activation='softmax', name='classic')(decoder_output)

    # 3. Angle type (3)
    # Angle (phi, theta, gamma) choosing
    angle = BatchNormalization()(decoder_output)
    angle = Dense(3, name='angle')(angle)

    if (output_style == 0):
        ## Concatenate the total output
        # (8 + 6 + 6 + 6 + 3)
        # (29)
        total = concatenate([gate, control, target, classic, angle], name='circuit')
        optimize_model = Model([circuit_input, hardware_input, decoder_input], total)
        optimize_model.compile(optimizer=Adam(0.001), loss='mse', metrics=['accuracy'])
        return optimize_model
    elif (output_style == 1):
        ## Concatenate the types
        # (8 + 6 + 6 + 6) + (3)
        # (26) + (3)
        total_type = concatenate([gate, control, target, classic], name='type')
        optimize_model = Model([circuit_input, hardware_input, decoder_input], [total_type, angle])
        optimize_model.compile(optimizer=Adam(0.0001),
                                 loss=['binary_crossentropy', 'mse'],
                                 loss_weights=[0.5, 0.5],
                                 metrics=['accuracy'])
        return optimize_model
    elif (output_style == 2):
        ## Seperated output
        # (8) + (6) + (6) + (6) + (3)
        optimize_model = Model([circuit_input, hardware_input, decoder_input], [gate, control, target, classic, angle])
        optimize_model.compile(optimizer=Adam(0.0001),
                                loss=['categorical_crossentropy',
                                    'categorical_crossentropy',
                                    'categorical_crossentropy',
                                    'categorical_crossentropy',
                                    'mse'],
                                loss_weights=[0.2, 0.2, 0.2, 0.2, 0.2],
                                metrics=['accuracy'])
        return optimize_model
    else:
        raise "You have to choose output style one of between [0, 1, 2]"

In [0]:
# TODO - Choose the output style 
# Model and dataset will be set automatically depends on the output style
"""
    output_style
    0 - concatenated
        (gate + control + target + classic + angle)
        [29]
    1 - type + angle
        (gate + control + target + classic) (angle)
        [26] + [3]
    2 - Seperated   
        (gate) (control) (target) (classic) (angle)
        [8] + [6] + [6] + [6] + [3]
"""
output_style = 0

model = optimize_circuit(circuit_dim,
                         circuit_timesteps,
                         env_dim,
                         latent_dim,
                         decoder_timesteps,
                         layout_n,
                         output_style=output_style)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
circuit_encode (InputLayer)     (None, 65, 29)       0                                            
__________________________________________________________________________________________________
lstm_3 (LSTM)                   (None, 64)           24064       circuit_encode[0][0]             
__________________________________________________________________________________________________
hardware (InputLayer)           (None, 70)           0                                            
__________________________________________________________________________________________________
concatenate_3 (Concatenate)     (None, 134)          0           lstm_3[0][0]                     
                                                                 hardware[0][0]             

### Training step 2 - Train the model
- Choose training hyperparameters
    - epochs
    - batch_size
    - shuffle
- Save the model and weights
- Load the weights for sequential train
- Load the weights for transfer learning from layout prediction model

#### Load the weight for sequential train
**Model should be already pre-built**

In [0]:
# TODO - Choose weight name to load
model_weight = "optimize_circuit.h5"

model.load_weights(os.path.join(weight_path, model_weight))
print("Loaded weight for sequential model")

#### Transfer learning from layout prediction model
**Loading the weight of layout prediction model**

In [0]:
# TODO - Choose weight name to load
enc_weight = "layout.h5"

model.load_weights(os.path.join(weight_path, enc_weight))
print("Loaded weight for transfer learning from layout")

#### Train

In [0]:
# Train hyperparameter
# TODO - Choose the hyperparameter
shuffle = True
epochs = 100
batch_size = 128

if (output_style == 0):
    model.fit([train_enc_in, train_hardware_in, train_dec_in],
          train_dec_out,
          verbose=1,
          shuffle=shuffle,
          epochs=epochs,
          batch_size=batch_size,  
          validation_data=([validation_enc_in,
                            validation_hardware_in,
                            validation_dec_in],
                           validation_dec_out))
elif (output_style == 1):
    model.fit([train_enc_in, train_hardware_in, train_dec_in],
            [train_type,
            train_angle],
            verbose=1,
            shuffle=shuffle,
            epochs=epochs,
            batch_size=batch_size,  
            validation_data=([validation_enc_in,
                            validation_hardware_in,
                            validation_dec_in],
                             [validation_type,
                              validation_angle]),)
elif (output_style == 2):
    model.fit([train_enc_in, train_hardware_in, train_dec_in],
            [train_gate,
            train_control,
            train_target,
            train_classic,
            train_angle],
            verbose=1,
            shuffle=shuffle,
            epochs=epochs,
            batch_size=batch_size,  
            validation_data=([validation_enc_in,
                            validation_hardware_in,
                            validation_dec_in],
                             [validation_gate,
                              validation_control,
                              validation_target,
                              validation_classic,
                              validation_angle]),)
else:
    raise "You have to choose output style one of between [0, 1, 2]"

In [0]:
# TODO - Choose the model & weight name
model_name = "optimize_circuit.json"
model_weight = "optimize_circuit.h5"

model_json = model.to_json()
with open(os.path.join(model_path, model_name), "w") as f:
    f.write(model_json)
print("Saved model to disk")    
model.save_weights(os.path.join(weight_path, model_weight))
print("Saved weight to disk")

Saved model to disk
Saved weight to disk


### Train step 3 - Generate the circuit and compare with the original optimized circuit

In [0]:
# gate type encode
gate_2_id = {'U1Gate': 0, 'U2Gate':1, 'U3Gate': 2, 'CnotGate': 3, 'Measure': 4, 'IdGate': 5, 'Barrier': 6, 'X': 7}
id_2_gate = dict((v, k) for k, v in gate_2_id.items())
# layout encode
# physical qubit to number
p2n = [0., 0.25, 0.5, 0.75, 1.]

In [0]:
#@title Compare the original and optimized circuit
def compare_results(index, original, predict):
    print("TEST SET %d" % index)
    print("=================ORIGINAL=================\n=================RESTORE==================\n")
    print("GATE NUMBER | GATE TYPE | CONTROL TARGET MEASURE | (THETA PIE GAMMA)\n")
    for i in range(len(original)):
        # predict
        p_gate = predict[i][:8]
        p_control= predict[i][8:14]
        p_target = predict[i][14:20]
        p_classic = predict[i][20:26]
        p_angle = predict[i][26:]

        p_gate = np.argmax(p_gate)
        p_control = np.argmax(p_control)
        p_target = np.argmax(p_target)
        p_classic = np.argmax(p_classic)

        # original
        o_gate = original[i][:8]
        o_control = original[i][8:14]
        o_target = original[i][14:20]
        o_classic = original[i][20:26]
        o_angle = original[i][26:]

        o_gate = np.argmax(o_gate)
        o_control = np.argmax(o_control)
        o_target = np.argmax(o_target)
        o_classic = np.argmax(o_classic)
        
        print("%-10s" % ("GATE : {}".format(i)), end="")
        print("%-8s" % id_2_gate[o_gate], end=" ")
        print("%-4s" % "[C:{}".format(o_control - 1 if o_control else ' '), end=" ")
        print("T:{}".format(o_target - 1 if o_target else ' '), end=" ")
        print("M:{}]".format(o_classic - 1 if o_classic else ' '), end=" ")
        print("(%.2f, %.2f, %.2f)" % (o_angle[0], o_angle[1], o_angle[2]))

        print("%-10s" % " ", end="")
        print("%-8s" % id_2_gate[p_gate], end=" ")
        print("%-4s" % "[C:{}".format(p_control - 1 if p_control else ' '), end=" ")
        print("T:{}".format(p_target - 1 if p_target else ' '), end=" ")
        print("M:{}]".format(p_classic - 1 if p_classic else ' '), end=" ")
        print("(%.2f, %.2f, %.2f)" % (p_angle[0], p_angle[1], p_angle[2]), end='\n\n')

In [0]:
#@title Predict circuit
def predict_circuit(index, output_style=0):
    enc_in = test_enc_in[index]
    hardware = test_hardware_in[index]
    dec_in = test_dec_in[index]
    if (output_style == 0):
        predict = model.predict([[enc_in], [hardware], [dec_in]])
        predict = predict[0]
    elif (output_style == 1):
        types, angles = model.predict([[enc_in], [hardware], [dec_in]])
        predict = np.hstack((types[0], angles[0]))
    elif (output_style == 2):
        gates, controls, targets, classics, angles = model.predict([[enc_in], [hardware], [dec_in]])
        predict = np.hstack((gates[0], controls[0], targets[0], classics[0], angles[0]))
    else:
        raise "You have to choose output style one of between [0, 1, 2]"
    compare_results(index, enc_in, predict)

In [0]:
# TODO - choose what test case you want to check

test_index = 2000
predict_circuit(test_index, output_style)